In [8]:
# !pip install -qU python-gdcm pydicom pylibjpeg
# !pip install -U pylibjpeg-libjpeg -v
# !pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg
# !pip install pydicom

# import include nesscessaryz
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil
from scipy import stats as sst
from Evaluate import *
from Setting_data import *

In [9]:
rsna = pd.read_csv('E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\RSNA.csv')
ddsm = pd.read_csv('E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM.csv')
cmmd = pd.read_csv('E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CMMD.csv')
cdd = pd.read_csv('E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\cdd_cesm.csv')

In [27]:
def SETTING(data):
    ev = Setting_data(data)
    ev.multi_process(list(data.patient_id.unique()))
    print(ev.meta_pro)
    return ev.meta_pro

In [31]:
def show(data, name):
    ev = Evaluate(data, name)
    ev.show()

In [32]:
cdd_new = SETTING(cdd)
cdd_new

e:\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Evalute_data\Setting_data.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['id_img_zip'] = None
e:\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Evalute_data\Setting_data.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.id_img_zip[i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'
e:\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Evalute_data\Setting_data.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

    patient_id       image_id view laterality  cancer
0         3CDD     P3-L-DM-CC   CC          L       1
1         3CDD    P3-L-DM-MLO  MLO          L       1
2         5CDD     P5-R-DM-CC   CC          R       0
3         5CDD    P5-R-DM-MLO  MLO          R       0
4         5CDD     P5-L-DM-CC   CC          L       0
..         ...            ...  ...        ...     ...
579     324CDD  P324-L-DM-MLO  MLO          L       0
580     325CDD   P325-L-DM-CC   CC          L       0
581     325CDD  P325-L-DM-MLO  MLO          L       0
582     326CDD   P326-L-DM-CC   CC          L       1
583     326CDD  P326-L-DM-MLO  MLO          L       1

[584 rows x 5 columns]


,patient_id,image_id,view,laterality,cancer
0,3CDD,P3-L-DM-CC,CC,L,1
1,3CDD,P3-L-DM-MLO,MLO,L,1
2,5CDD,P5-R-DM-CC,CC,R,0
3,5CDD,P5-R-DM-MLO,MLO,R,0
4,5CDD,P5-L-DM-CC,CC,L,0
...,...,...,...,...,...
579,324CDD,P324-L-DM-MLO,MLO,L,0
580,325CDD,P325-L-DM-CC,CC,L,0
581,325CDD,P325-L-DM-MLO,MLO,L,0
582,326CDD,P326-L-DM-CC,CC,L,1


In [33]:
len(cdd_new)

584

In [34]:
show(cdd_new, 'cdd')

Dataset cdd 
 Số lượng image: 584 
 Số luong patient: 199
So luong CC và MLO
view
CC     292
MLO    292
Name: count, dtype: int64
So luong L và R
view  laterality
CC    L             149
      R             143
MLO   L             149
      R             143
dtype: int64
So luong img cancer: 297
So luong img no cancer: 287


In [36]:
inbreast = pd.read_csv(r"E:\WORKBASE\Project-rsna-breast-cancer-detection\Inbreasts\INbreast Release 1.0\INbreast.csv")
inbreast

,Patient ID,Patient age,Laterality,View,Acquisition date,File Name,ACR,Bi-Rads
0,removed,removed,R,CC,201001,22678622,4,1
1,removed,removed,L,CC,201001,22678646,4,3
2,removed,removed,R,MLO,201001,22678670,4,1
3,removed,removed,L,MLO,201001,22678694,4,3
4,removed,removed,R,CC,201001,22614074,2,5
...,...,...,...,...,...,...,...,...
405,removed,removed,L,MLO,201001,22613848,2,1
406,removed,removed,L,MLO,200802,24055725,4,2
407,removed,removed,R,MLO,200802,24055752,4,2
408,removed,removed,L,CC,200802,24055779,4,2
